In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

# Importando a lista de Ações e a Chave API 

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [3]:
from secrets import IEX_CLOUD_API_TOKEN
# Adquira suas chave API no site: https://iexcloud.io

# Fazendo o primeiro uso da API
Vamos pegar o preço da ação da Apple e o valor de mercado da mesma

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

In [5]:
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'K)NRQAE OTLEDM AASLELSS/GNA TCBG(', 'calculationPrice': 'tops', 'open': None, 'openTime': None, 'openSource': 'iacfloif', 'close': None, 'closeTime': None, 'closeSource': 'cfifalio', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 137.69, 'latestSource': 'IEX real time price', 'latestTime': '12:59:51 PM', 'latestUpdate': 1671847099388, 'latestVolume': None, 'iexRealtimePrice': 136.87, 'iexRealtimeSize': 105, 'iexLastUpdated': 1652620681414, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 139.78, 'previousVolume': 106717291, 'change': 0.88, 'changePercent': 0.00645, 'volume': None, 'iexMarketPercent': 0.017570093431438855, 'iexVolume': 800467, 'avgTotalVolume': 122828403, 'ie

In [6]:
price = data['latestPrice']
market_cap = data['marketCap']

In [10]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
     index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,137.69,2344564712537,N/A


# Fazendo o mesmo procedimento com nossa lista de ações
A próxima coisa a se fazer é adicionar os dados desejados em um pandas DataFrame

In [39]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index = my_columns
        ),
        ignore_index = True
    )

In [40]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,124,37479121653,N/A
1,AAL,17.3,10434641818,N/A
2,AAP,151.63,10153034787,N/A
3,AAPL,136.26,2264278468859,N/A
4,ABBV,107.5,184557693604,N/A
...,...,...,...,...
500,YUM,101.63,30834057566,N/A
501,ZBH,158.7,33356444874,N/A
502,ZBRA,402.31,21556268579,N/A
503,ZION,45.65,7593089658,N/A


# Batch API para melhorar a Performance
O código acima fez requests via HTTP, que é extremamente devagar, uma vez que é necessário fazer uma request para cada loop.
O IEX Cloud, que estamos usando como API, limita a batck API em 100 tickers por request. Por cotna disso, vamos separar as ações do SPX em grupos de 100

In [11]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [12]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index= True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,129.057,38117535295,N/A
1,AAL,17.158,10570878822,N/A
2,AAP,163.140,11047688057,N/A
3,AAPL,135.058,2324144124980,N/A
4,ABBV,106.160,188400182708,N/A
...,...,...,...,...
500,YUM,108.300,33768685759,N/A
501,ZBH,161.040,33668125713,N/A
502,ZBRA,413.850,22341265742,N/A
503,ZION,48.350,7727768829,N/A


# Calculando o número de ações para comprar

In [13]:
portfolio_size = input('Enter the value of your portfolio: ')

teste = 0
while teste == 0:
    try:
        val = float(portfolio_size)
        teste = 1
    except:
        print('Please enter an integer')
        portfolio_size = input('Enter the value of your portfolio: ')
        teste = 0


Enter the value of your portfolio: dasdassd
Please enter an integer
Enter the value of your portfolio: asdasds
Please enter an integer
Enter the value of your portfolio: asdasdas
Please enter an integer
Enter the value of your portfolio: dasdasdas
Please enter an integer
Enter the value of your portfolio: 1000


In [14]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,129.057,38117535295,0
1,AAL,17.158,10570878822,0
2,AAP,163.140,11047688057,0
3,AAPL,135.058,2324144124980,0
4,ABBV,106.160,188400182708,0
...,...,...,...,...
500,YUM,108.300,33768685759,0
501,ZBH,161.040,33668125713,0
502,ZBRA,413.850,22341265742,0
503,ZION,48.350,7727768829,0


# Excel Output
O código abaixo permite que todo esse código seja exportado para uma planilha formatada no Excel. Sobre a formatação, é possível fo

In [106]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [107]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [108]:
#writer.sheets['Recommended Trades'].set_column('A:A',18, string_format )
#writer.sheets['Recommended Trades'].set_column('B:B',18, string_format )
#writer.sheets['Recommended Trades'].set_column('C:C',18, string_format )
#writer.sheets['Recommended Trades'].set_column('D:D',18, string_format )
#writer.save()

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

In [109]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
writer.save()